# RNN to Classify Reuters Topics

In this project, I will be training a recurrent neural network to Classify Reuters newswires into 46 Topics.

Dataset of 11,228 newswires from Reuters, labeled over 46 topics. Each wire is encoded as a sequence of word indexes.

In [ ]:
%tensorflow_version 2.8.0
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.autograph.set_verbosity(0)

from __future__ import print_function

!pip install numpy
import numpy as np
import keras
keras.__version__

import random
seed = 32
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.8.0`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Loading the Reuters Dataset


In [ ]:
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

max_words = 10000 # Number of words to consider as features

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words, test_split=0.2)
word_index = reuters.get_word_index(path="reuters_word_index.json")

Loading data...
565248/550378 [==============================] - 0s 0us/step


In [ ]:
# Pad train and test sequences
from keras import preprocessing
maxlen=200
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
print('input_train shape:', x_train.shape)
print('input_test shape:', x_test.shape)

input_train shape: (8982, 200)
input_test shape: (2246, 200)


In [ ]:
# Converting labels into one hot encodings
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

## RNN and LSTM


### Single layer RNN model with 128 output units for 10 epochs

In [ ]:
#Training a RNN model
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(10000, embedding_size))
model.add(SimpleRNN(128))
model.add(Dense(46, activation='softmax'))

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['acc']
)
history = model.fit(
    x_train,
    one_hot_train_labels,
    epochs=10,
    batch_size=128,
    validation_split=0.2
)

Epoch 1/10
57/57 [==============================] - 12s 200ms/step - loss: 2.4315 - acc: 0.3669 - val_loss: 2.4196 - val_acc: 0.3484
Epoch 2/10
57/57 [==============================] - 11s 196ms/step - loss: 2.1094 - acc: 0.4459 - val_loss: 2.2822 - val_acc: 0.4090
Epoch 3/10
57/57 [==============================] - 11s 197ms/step - loss: 1.9632 - acc: 0.4994 - val_loss: 2.3266 - val_acc: 0.4085
Epoch 4/10
57/57 [==============================] - 11s 196ms/step - loss: 1.8349 - acc: 0.5314 - val_loss: 2.2921 - val_acc: 0.3996
Epoch 5/10
57/57 [==============================] - 11s 192ms/step - loss: 1.4897 - acc: 0.6278 - val_loss: 2.2082 - val_acc: 0.4463
Epoch 6/10
57/57 [==============================] - 12s 213ms/step - loss: 1.2663 - acc: 0.6863 - val_loss: 2.3082 - val_acc: 0.4290
Epoch 7/10
57/57 [==============================] - 11s 198ms/step - loss: 0.9997 - acc: 0.7574 - val_loss: 2.4336 - val_acc: 0.4040
Epoch 8/10
57/57 [==============================] - 11s 199ms/step - 

In [ ]:
# Evaluate RNN model on test set
model.evaluate(x_test, one_hot_test_labels)

71/71 [==============================] - 1s 16ms/step - loss: 2.6717 - acc: 0.4065


[2.6717309951782227, 0.4065004587173462]

### Single layer LSTM model with 128 output units for 20 epochs

In [ ]:
# Training a LSTM model
from keras.layers import LSTM, Dropout

model = Sequential()
model.add(Embedding(10000, embedding_size))
model.add(LSTM(128))
model.add(Dense(46, activation='softmax'))

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['acc']
)
history = model.fit(
    x_train,
    one_hot_train_labels,
    epochs=20,
    batch_size=128,
    validation_split=0.2
)

Epoch 1/20
57/57 [==============================] - 5s 27ms/step - loss: 2.4388 - acc: 0.3677 - val_loss: 3.0421 - val_acc: 0.3523
Epoch 2/20
57/57 [==============================] - 1s 18ms/step - loss: 2.0216 - acc: 0.4860 - val_loss: 1.9356 - val_acc: 0.4880
Epoch 3/20
57/57 [==============================] - 1s 19ms/step - loss: 1.7632 - acc: 0.5388 - val_loss: 1.7474 - val_acc: 0.5442
Epoch 4/20
57/57 [==============================] - 1s 25ms/step - loss: 1.6352 - acc: 0.5756 - val_loss: 1.6710 - val_acc: 0.5899
Epoch 5/20
57/57 [==============================] - 1s 20ms/step - loss: 1.5365 - acc: 0.5968 - val_loss: 1.6385 - val_acc: 0.5838
Epoch 6/20
57/57 [==============================] - 1s 18ms/step - loss: 1.4032 - acc: 0.6315 - val_loss: 1.5167 - val_acc: 0.6210
Epoch 7/20
57/57 [==============================] - 1s 18ms/step - loss: 1.2755 - acc: 0.6632 - val_loss: 1.4604 - val_acc: 0.6333
Epoch 8/20
57/57 [==============================] - 1s 18ms/step - loss: 1.1469 - a

In [ ]:
# Evaluating LSTM model on test set
model.evaluate(x_test, one_hot_test_labels)

71/71 [==============================] - 0s 7ms/step - loss: 1.7638 - acc: 0.6621


[1.7637970447540283, 0.6620659232139587]

## Using a Pre-trained Word Embedding




In [ ]:
# Training a LSTM model using pretrained embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


--2022-06-07 22:02:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-07 22:02:45--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-07 22:02:45--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
import os
path_to_glove_file = os.path.join(
    "/content/glove.6B.200d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
# Prepare embedding matrix
num_tokens = 35000
embedding_dim = 200
hits = 0
misses = 0
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))
# len(embedding_matrix)

Converted 24975 words (6004 misses)


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)


In [ ]:
model = Sequential()
model.add(Embedding(10000, embedding_size))
model.add(LSTM(128))
model.add(Dense(46, activation='softmax'))

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['acc']
)
history = model.fit(
    x_train,
    one_hot_train_labels,
    epochs=20,
    batch_size=128,
    validation_split=0.2
)

Epoch 1/20
57/57 [==============================] - 3s 27ms/step - loss: 2.4143 - acc: 0.3905 - val_loss: 2.3444 - val_acc: 0.4101
Epoch 2/20
57/57 [==============================] - 1s 18ms/step - loss: 2.0059 - acc: 0.4909 - val_loss: 2.3160 - val_acc: 0.4791
Epoch 3/20
57/57 [==============================] - 1s 18ms/step - loss: 1.7659 - acc: 0.5421 - val_loss: 1.6846 - val_acc: 0.5626
Epoch 4/20
57/57 [==============================] - 1s 19ms/step - loss: 1.6453 - acc: 0.5754 - val_loss: 1.8841 - val_acc: 0.5364
Epoch 5/20
57/57 [==============================] - 1s 18ms/step - loss: 1.5199 - acc: 0.5882 - val_loss: 1.8081 - val_acc: 0.5743
Epoch 6/20
57/57 [==============================] - 1s 18ms/step - loss: 1.4357 - acc: 0.6186 - val_loss: 1.5069 - val_acc: 0.6121
Epoch 7/20
57/57 [==============================] - 1s 18ms/step - loss: 1.2833 - acc: 0.6612 - val_loss: 1.5037 - val_acc: 0.6294
Epoch 8/20
57/57 [==============================] - 1s 18ms/step - loss: 1.1667 - a

In [ ]:
# Evaluating LSTM model on test set
model.evaluate(x_test, one_hot_test_labels)

71/71 [==============================] - 1s 11ms/step - loss: 1.7596 - acc: 0.6634


[1.7595865726470947, 0.6634016036987305]